# IMPORTACION
se realiza la importaion de las librerias que usaremos para porder procesar los data set 
* pandas para leer y manipular archivos
* numpy para hacer los calculos 
* matplotlib para graficar 
* sklearn tareas de aprendizaje automatico

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn 


# realizacion de limpieza 
 se eliminan los datos que no sumarian al análisis ya sean columnas filas y se renombran algunas columnas 

In [ ]:
# Cargar el archivo Excel
dat_excel = pd.read_excel(
    '14_3_02_Establecimientos_industriales_rama-1.xlsx',
    engine='openpyxl',
    skiprows=2
)

# Pasos de limpieza iniciales que ya tenías
dat_excel = dat_excel.iloc[:-6] # Elimina las últimas 6 filas
columns_to_drop = ['Otras', 'Total', 'Electrónicas', 'Mecánica','Confeccionistas'] # Columnas a eliminar
dat_excel = dat_excel.drop(columns=columns_to_drop) # Elimina las columnas especificadas
dat_excel = dat_excel.rename(columns={'Unnamed: 1': 'Mes', 'index': 'indice'}) # Renombra columnas
dat_excel = dat_excel.reset_index(drop=True) # Resetea el índice y evita crear una nueva columna de índice
dat_excel = dat_excel.replace('...', 0) # Reemplaza '...' por 0

# --- Obtener el primer año válido desde la columna 'Año / Mes' ---
primer_anio = None
for valor in dat_excel['Año / Mes']:
    if str(valor).isdigit() and len(str(valor)) == 4: # Busca un valor numérico de 4 dígitos (año)
        primer_anio = int(valor)
        break

if primer_anio is None:
    primer_anio = 2001 # Predeterminado si no se encuentra ningún año

# --- Eliminar las filas que contienen solo el año ---
# Esto filtra las filas donde 'Año / Mes' es un año de 4 dígitos
dat_excel = dat_excel[~dat_excel['Año / Mes'].apply(
    lambda x: str(x).isdigit() and len(str(x)) == 4
)].copy()

# --- Asignar el año cambiando cada 12 filas ---
# Crea una lista de años que se repite cada 12 meses
num_filas = len(dat_excel)
anios = [primer_anio + (i // 12) for i in range(num_filas)]
dat_excel['Año'] = anios # Asigna la nueva columna 'Año'

# Eliminar columna original de temporalidad
dat_excel = dat_excel.drop(columns=['Año / Mes'])

# Mover la columna 'Año' a la segunda posición por ahora (se reordenará de nuevo para el índice)
año_col = dat_excel.pop('Año')
dat_excel.insert(1, 'Año', año_col)

# Convertir columnas específicas a tipo entero
columns_to_convert_to_int = [ 'Plásticas', 'Textiles', 'Pesqueras']
for col in columns_to_convert_to_int:
    dat_excel[col] = pd.to_numeric(dat_excel[col], errors='coerce').fillna(0).astype(int)


# --- NUEVAS TRANSFORMACIONES EMPIEZAN AQUÍ ---

# Aunque no la vamos a mantener, el mapeo del mes puede ser útil para depuración o si se decidiera usar en el futuro
month_mapping = {
    'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,
    'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
}
dat_excel['Mes'] = dat_excel['Mes'].map(month_mapping)
dat_excel['Mes'] = dat_excel['Mes'].fillna(0).astype(int) # Rellena NaN con 0 y luego convierte a int


# 2. Añadir la columna 'Indice'
dat_excel.insert(0, 'Indice', range(1, 1 + len(dat_excel)))

# Ahora, reordenar 'Año' para que esté después de 'Indice'
cols = dat_excel.columns.tolist()
año_col_name = 'Año'
if año_col_name in cols:
    cols.remove(año_col_name)
    cols.insert(1, año_col_name)
    dat_excel = dat_excel[cols]

# 3. Sumar las columnas especificadas
dat_excel['Suma Industrias'] = dat_excel['Plásticas'] + dat_excel['Textiles'] + dat_excel['Pesqueras']

# --- NUEVO PASO: ELIMINAR LA COLUMNA 'Mes' ---
if 'Mes' in dat_excel.columns:
    dat_excel = dat_excel.drop(columns=['Mes'])


# --- NUEVAS TRANSFORMACIONES TERMINAN AQUÍ ---

# Resultado Final
num_filas, num_columnas = dat_excel.shape
#print(f"\nNúmero de filas: {num_filas}")
#print(f"Número de columnas: {num_columnas}")



# lectura y modificacion del segundo datase 

In [ ]:
dat_excel2 = pd.read_excel('21_1_01_Remuneracion-promedio-de-los-trabajadores-registrados-del-sector-privado-segun-rama-de-actividad.xlsx', engine='openpyxl',
   skiprows=2)
Gastos_df = dat_excel2
#Gastos_df.head() # Para verificar que se cargó correctamente

Gastos_df= Gastos_df.rename(columns={'Unnamed: 1': 'periodo'})
Gastos_df= Gastos_df.drop(columns=['Rama de Actividad'])
Gastos_df= Gastos_df.rename(columns={'periodo': 'indice'})

Gastos_df.head()					




# trasponer
se traspone los datos para poder trabajar sobre ellos en el cual queda como resultado las actividades como encabezado de columnas y los años en las filas 

In [ ]:
# Transponer
Gastos_df = Gastos_df.T
Gastos_df.columns = Gastos_df.iloc[0]  # Asigna la primera fila como encabezado
Gastos_df = Gastos_df.drop(Gastos_df.index[0])  # Elimina esa primera fila
Gastos_df= Gastos_df.reset_index()
Gastos_df= Gastos_df.rename(columns={'index': 'periodos'})

Gastos_df.head()

se segrega los campos que seran utilizados para el analisis

In [ ]:
import pandas as pd
import numpy as np # Necesitamos numpy para np.nan

# Suponiendo que 'Gastos_df' ya está cargado con la estructura que se ve en la imagen.
# Si no, necesitarías cargarlo primero (ejemplo):
# Gastos_df = pd.read_excel('tu_archivo_de_gastos.xlsx', engine='openpyxl')
# Asegúrate de que 'periodo' sea la columna que quieres renombrar a 'indice'

# Primero, vamos a renombrar 'periodo ' a 'indice' lo antes posible
# para que las operaciones subsiguientes usen el nombre correcto.
# Asegúrate de que 'periodo ' (con espacio al final) es el nombre correcto en tu DF.
#if 'periodo ' in Gastos_df.columns:
#    Gastos_df = Gastos_df.rename(columns={'periodo ': 'indice'})
#else:
 #   print("Advertencia: La columna 'periodo ' no se encontró para renombrar a 'indice'.")


# --- Eliminar filas desde el índice 0 al 71 (o hasta el 72 si era el 73 en range) ---
# Si tu objetivo era hasta el índice 72 inclusive (total 73 filas), debería ser range(0, 73).
# Ajusta según tu necesidad exacta.
indices_a_eliminar = range(0, 72)
Gastos_df = Gastos_df.drop(index=indices_a_eliminar)
Gastos_df = Gastos_df.reset_index(drop=True) # Reiniciar el índice para que sea consecutivo


# --- Lista de columnas a eliminar ---
# Ahora, 'Confecciones' ya está aquí para ser eliminada explícitamente.
columns_to_drop_initial = [ # Renombré para claridad
    'AGRICULTURA, GANADERIA, CAZA Y SILVICULTURA',
    ' Agricultura y ganadería ',
    ' Silvicultura, extracción de madera ',
    'PESCA Y SERVICIOS CONEXOS',
    'EXPLOTACION  DE  MINAS  Y  CANTERAS',
    ' Extracción de petróleo crudo y gas natural ',
    ' Extracción de minerales metalíferos ',
    ' Explotación de otras minas y canteras ',
    'INDUSTRIA MANUFACTURERA',
    ' Alimentos ',
    ' Tabaco ',
    ' Cuero y calzado ',
    ' Madera ',
    ' Papel ',
    ' Edición ',
    ' Productos de petróleo ',
    ' Productos químicos ',
    ' Otros minerales no metálicos ',
    ' Metales comunes ',
    ' Otros productos de metal ',
    ' Maquinaria y equipo ',
    ' Maquinaria de oficina ',
    ' Aparatos eléctricos ',
    ' Radio y televisión ',
    ' Instrumentos médicos ',
    ' Automotores ',
    ' Otros equipo de transporte ',
    ' Muebles ',
    ' Reciclamiento de desperdicios y desechos ',
    'ELECTRICIDAD, GAS Y AGUA',
    ' Electricidad, gas y agua ',
    ' Captación, depuración y distribución de agua ',
    'CONSTRUCCION',
    ' Construcción ',
    'COMERCIO AL POR MAYOR Y AL POR MENOR',
    ' Vta. y reparación de vehículos. vta. por menor de combustible ',
    ' Comercio al por mayor ',
    ' Comercio al por menor ',
    'HOTELERIA Y RESTAURANTES ',
    ' Servicios de hotelería y restaurantes ',
    'SERVICIOS DE TRANSPORTE, DE ALMACENAMIENTO Y DE COMUNICACIONES',
    ' Transporte ferroviario y automotor y por tuberías ',
    ' Transporte marítimo y fluvial ',
    ' Transporte aéreo de cargas y de pasajeros ',
    ' Manipulación de carga, almacenamiento y depósito ',
    ' Telecomunicaciones y correos ',
    'INTERMEDIACION FINANCIERA Y OTROS SERVICIOS FINANCIEROS ',
    ' Intermediación financiera y otros servicios financieros ',
    ' Seguros ',
    ' Servicios auxiliares a la actividad financiera ',
    'SERVICIOS INMOBILIARIOS, EMPRESARIALES Y DE ALQUILER',
    ' Servicios inmobiliarios ',
    ' Alquiler de equipo de transporte y de maquinaria ',
    ' Actividades de informática ',
    ' Investigación y desarrollo ',
    ' Servicios jurídicos, contables y otros servicios a empresas ',
    ' Agencias de empleo eventual ',
    'ENSEÑANZA',
    ' Enseñanza ',
    'SERVICIOS SOCIALES Y DE SALUD',
    ' Servicios sociales y de salud ',
    'SERVICIOS COMUNITARIOS, SOCIALES Y PERSONALES N.C.P.',
    ' Eliminación de desperdicios ',
    ' Servicios de organizaciones empresariales ',
    ' Servicios culturales, deportivos y de esparcimiento ',
    'Servicios n.c.p.',
    ' Confecciones '
]

# Elimina las columnas especificadas en columns_to_drop_initial
Gastos_df = Gastos_df.drop(columns=columns_to_drop_initial, errors='ignore')

# --- Eliminar columnas que sean completamente NaN o que pandas haya nombrado 'Unnamed: X' y estén vacías ---
# Este paso sigue siendo muy útil.
Gastos_df = Gastos_df.dropna(axis=1, how='all')


# --- PASO ADICIONAL: Eliminar columnas por su nombre específico 'NaN' o 'Unnamed: X' ---
# Esto es para capturar las columnas que pandas nombró 'NaN' o 'Unnamed: X'
# y que no fueron eliminadas por dropna(how='all') porque quizás no estaban completamente vacías,
# o porque el nombre 'NaN' es un objeto especial (numpy.nan).

columns_to_drop_by_name = []
for col in Gastos_df.columns:
    # Caso 1: La columna se llama literalmente 'NaN' (como string)
    if isinstance(col, str) and col == 'NaN':
        columns_to_drop_by_name.append(col)
    # Caso 2: La columna se llama como un objeto NaN (numpy.nan)
    elif pd.isna(col): # pd.isna() funciona para np.nan o valores flotantes NaN
        columns_to_drop_by_name.append(col)
    # Caso 3: La columna se llama 'Unnamed: X' (común en Excel)
    elif isinstance(col, str) and col.startswith('Unnamed:'):
        columns_to_drop_by_name.append(col)

if columns_to_drop_by_name: # Si hay columnas para eliminar
    print(f"Eliminando columnas con nombres problemáticos: {columns_to_drop_by_name}")
    Gastos_df = Gastos_df.drop(columns=columns_to_drop_by_name)


# --- Columnas que vas a sumar ---
# Asegúrate de que los nombres coinciden EXACTAMENTE con los de tu DataFrame después de todas las eliminaciones.
columns_to_sum = [
    ' Pesca y actividades relacionadas con la pesca ',
    ' Productos textiles ',
    ' Productos de caucho y plástico ',]
    
# --- PASO CLAVE: Convertir las columnas a tipo numérico y manejar errores ---
for col in columns_to_sum:
    if col in Gastos_df.columns:
        # Convertir a numérico, forzando errores a NaN, luego rellenar NaN con 0 y convertir a int.
        Gastos_df[col] = pd.to_numeric(Gastos_df[col], errors='coerce').fillna(0).astype(int)
    else:
        print(f"Advertencia: La columna '{col}' no se encontró en el DataFrame y no se pudo procesar para la suma.")

# Realiza la suma.
Gastos_df['Suma de gastos'] = Gastos_df[columns_to_sum].sum(axis=1, skipna=True)

# --- SEGMENTO PARA AGREGAR UN NUEVO ÍNDICE NUMÉRICO CONSECUTIVO AL FINAL ---

# Asegúrate de que este código se ejecuta después de TODAS las eliminaciones de filas
# y cualquier otra operación que pueda cambiar el número de filas o su orden.

# 1. Primero, asegúrate de que el índice del DataFrame sea numérico consecutivo (0, 1, 2...).
# Esto ya se hace con Gastos_df = Gastos_df.reset_index(drop=True) en tu código.
# Si lo has eliminado por alguna razón, o si necesitas un nuevo reinicio, colócalo aquí:
Gastos_df = Gastos_df.reset_index(drop=True)

# 2. Crea una nueva columna llamada 'indice' basada en el índice actual del DataFrame.
# El índice de un DataFrame de pandas es un objeto Index. Lo convertimos a una lista o Serie
# para asignarlo como una nueva columna.
Gastos_df['indice'] = Gastos_df.index + 1 #si quieres que empiece en 1

# 3. Opcional pero recomendado: Mover la nueva columna 'indice' al principio del DataFrame.
# Esto hace que sea más fácil de ver y usar.
cols = Gastos_df.columns.tolist()
if 'indice' in cols: # Asegurarse de que 'indice' existe antes de mover
    cols.remove('indice')
    Gastos_df = Gastos_df[['indice'] + cols]

# --- FIN DEL SEGMENTO ---


## Vista preliminar de los data set 


In [ ]:
Gastos_df.to_csv('Gastos.csv', index=False)
dat_excel.to_csv('Establecimientos.csv', index=False)

Gastos_df.head(15)

In [ ]:
dat_excel.head(15)